Studying the cases in the PuLP documentation - continued

# Beer Distribution (Transportation) Problem

- Distribute from **2 warehouses** to **5 bars**
- Warehouses (supply nodes) - {A: 1000 cases, B: 4000 cases}
- Bars (demand nodes) require - {1: 500, 2: 900, 3: 1800, 4: 200, 5: 700}
- Goal is to **minimize transportation costs, which warehouse should supply which bar?**
- Since total supply > total demand, put the surplus into a dummy demand node D
    - Putting into D is the equivalent of keeping it in the warehouse
    
**Note:** If a transportation problem has excess supply, you can balance it with putting the surplus into a dummy demand node. A solution to an unbalanced problem w/ excess demand is infeasible.

**Corollary:** If we had 4000 supply and 4100 demand, we'd create a dummy supply node (a.k.a competitor) with 100 cases.

## Data

**Supply:** A: 1000, B: 4000

**Demand:** 1: 500, 2: 900, 3: 1800, 4: 200, 5: 700, D: 900


**Cost per case**

| From warehouse to bar | A | B |
| --------- | - | - |
| 1 | 2 | 3 |
| 2 | 4 | 1 |
| 3 | 5 | 3 |
| 4 | 2 | 2 |
| 5 | 1 | 3 |
| D | 0 | 0 |

- Cost of storing the surplus is 0

## Problem Formulation

### Decision variables

- Edges A1..A5 and B1..B5
    - A1 is the # of cases to be sent from A to 1
    - Edges are nonnegative integers, cases can't be separated

### Objective function
- Minimize total cost
    - **min** `sum(cost_per_case(wh, bar) * cases_supplied(wh, bar) for each wh and bar`

### Constraints

- `sum(A1, ..., A5, AD) <= 1000` - A can't supply more than its capacity
- `sum(B1, ..., B5, BD) <= 4000` - B can't supply more than its capacity
- `Ai + Bi == demand(i) for each bar i` - Each bar's demand must be fulfilled (total supply is enough for that)

In [1]:
import pulp

warehouses = ["A", "B"]
supply = {"A": 1000, "B": 4000}

bars = ["1", "2", "3", "4", "5", "D"]
demand = {"1": 500, "2": 900, "3": 1800, "4": 200, "5": 700, "D": 900}

costs = {
    "A": {"1": 2, "2": 4, "3": 5, "4": 2, "5": 1, "D": 0},
    "B": {"1": 3, "2": 1, "3": 3, "4": 2, "5": 3, "D": 0}
}

routes = [(wh, bar) for wh in warehouses for bar in bars]

beer_dist = pulp.LpProblem("Beer_Distribution_Problem", pulp.LpMinimize)

# decision vars
cases_supplied = pulp.LpVariable.dicts("Route", (warehouses, bars), 0, None, pulp.LpInteger)

# objective func
beer_dist += pulp.lpSum([cases_supplied[wh][bar] * costs[wh][bar] for wh, bar in routes])

# constraints

# warehouses can't supply more than their capacities
for wh in warehouses:
    beer_dist += pulp.lpSum(cases_supplied[wh][bar] for bar in bars) <= supply[wh]

# bars' demands must be fulfilled
for bar in bars:
    beer_dist += pulp.lpSum(cases_supplied[wh][bar] for wh in warehouses) == demand[bar]
    
beer_dist.solve()
print("Status:", pulp.LpStatus[beer_dist.status])

for v in beer_dist.variables():
    print(v.name, "=", v.varValue)
    
print("Total transportation cost = ", pulp.value(beer_dist.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_A_D = 0.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Route_B_D = 900.0
Total transportation cost =  8600.0


# Two Stage Production Planning Problem

Goal: Maximize profit

GTC produces wrenches and pliers, subject to steel, machine capabilities (molding and assembly), labor, demand. Uncertain variables are expressed thru "scenarios"

Stage 1: Determine how much steel to purchase

Find out how much capacity is available and the revenue per unit of wrenches and pliers

Stage 2: Determine how many wrenches and pliers to produce

Four possible scenarios, each with equal probability

## Data

In [2]:
products = ["wrenches", "pliers"]
price = [130, 100]
steel = [1.5, 1]
molding = [1, 1]
assembly = [0.3, 0.5]
cap_steel = 27
cap_molding = 21
LB = [0, 0]
capacity_ub = [15, 16]
steel_price = 58

scenarios = [0, 1, 2, 3]
p_scenario = [0.25, 0.25, 0.25, 0.25]
wrench_earnings = [160, 160, 90, 90]
plier_earnings = [100, 100, 100, 100]
cap_assembly = [8, 10, 8, 10]

production = [(j, i) for j in scenarios for i in products]
price_scenario = [[wrench_earnings[j], plier_earnings[j]] for j in scenarios]
price_items = [item for sublist in price_scenario for item in sublist]

price_dict = dict(zip(production, price_items))
capacity_dict = dict(zip(products, capacity_ub * 4))
steel_dict = dict(zip(products, steel))
molding_dict = dict(zip(products, molding))
assembly_dict = dict(zip(products, assembly))

## Problem Formulation

In [3]:
prod_plan = pulp.LpProblem("Prod_Planning", pulp.LpMaximize)

### Decision variables

`w_i` - Production of wrenches in scenario i

`pl_i` - Production of pliers in scenario i

`steel_purchased`

In [4]:
production_vars = pulp.LpVariable.dicts(
    "production", (scenarios, products), 0, None, pulp.LpInteger
)
production_vars

{0: {'wrenches': production_0_wrenches, 'pliers': production_0_pliers},
 1: {'wrenches': production_1_wrenches, 'pliers': production_1_pliers},
 2: {'wrenches': production_2_wrenches, 'pliers': production_2_pliers},
 3: {'wrenches': production_3_wrenches, 'pliers': production_3_pliers}}

In [5]:
steel_purchased = pulp.LpVariable("steel_purchased", 0, None)

### Objective function
Profit: Revenue - Expenses

Expenses: `steel_purchased * steel_price`

Revenue: `sum(0.25 * price[scenario][item] * production_of[scenario][item] for each scenario, item)`

In [6]:
# Add obj func to problem

prod_plan += pulp.lpSum([p_scenario[scenario] * price_dict[scenario, item] * production_vars[scenario][item]
             for scenario, item in production]) - steel_price * steel_purchased

### Constraints

Don't exceed steel, molding, assembly capacities and the production upper bound

In [7]:
for scenario in scenarios:
    prod_plan += pulp.lpSum(
        [steel_dict[product] * production_vars[scenario][product] for product in products]
    ) <= steel_purchased
    
    prod_plan += pulp.lpSum(
        [molding_dict[product] * production_vars[scenario][product] for product in products]
    ) <= cap_molding
    
    prod_plan += pulp.lpSum(
        [assembly_dict[product] * production_vars[scenario][product] for product in products]
    ) <= cap_assembly[scenario]
    
    for product in products:
        prod_plan += production_vars[scenario][product] <= capacity_dict[product]

In [8]:
prod_plan.solve()
print("Status:", pulp.LpStatus[prod_plan.status])

for v in prod_plan.variables():
    print(v.name, "=", v.varValue)
production = [v.varValue for v in prod_plan.variables()]

print("Profit = ", pulp.value(prod_plan.objective))

Status: Optimal
production_0_pliers = 5.0
production_0_wrenches = 15.0
production_1_pliers = 5.0
production_1_wrenches = 15.0
production_2_pliers = 8.0
production_2_wrenches = 13.0
production_3_pliers = 16.0
production_3_wrenches = 5.0
steel_purchased = 27.5
Profit =  860.0
